In [1]:
import os
import sys
from tqdm import tqdm
import pickle

sys.path.append("./loaders")

In [2]:
# from ChebiLoader import ChebiLoader
# chebi = ChebiLoader("/local/sps-local/chebi/chebi.owl")

# from BC5CDRLoader import BC5CDRLoader
# cdr = BC5CDRLoader("./assets/train/BC5CDR/", chebi)

# with open("./assets/cdrloader.pkl", "wb") as f:
#     pickle.dump((cdr.spans, cdr.labels), f)

# print("saved prepared bc5cdr datasets")


In [3]:
# from NLMChemLoader import NLMChemLoader
# nlm = NLMChemLoader("./assets/train/NLM_Chem_corpus/", chebi)
# with open("./assets/nlmloader.pkl", "wb") as f:
#     pickle.dump((nlm.spans, nlm.labels), f)

# print("saved prepared nlm datasets")

In [4]:
# from CraftLoader import CraftLoader
# craft = CraftLoader("./assets/train/CRAFT/", chebi)
# with open("./assets/craftloader.pkl", "wb") as f:
#     pickle.dump((craft.spans, craft.labels), f)

# print("saved prepared craft datasets")

In [5]:
cdr_spans = []
cdr_labels = []

nlm_spans = []
nlm_labels = []

craft_spans = []
craft_labels = []


with open("./assets/cdrloader.pkl", "rb") as f:
    (cdr_spans, cdr_labels) = pickle.load(f)
print("cdr-spans:", len(cdr_spans), "cdr-labels:", len(cdr_labels))

with open("./assets/nlmloader.pkl", "rb") as f:
    (nlm_spans, nlm_labels) = pickle.load(f)
print("nlm-spans:", len(nlm_spans), "nlm-labels:", len(nlm_labels))

with open("./assets/craftloader.pkl", "rb") as f:
    (craft_spans, craft_labels) = pickle.load(f)
print("craft-spans:", len(craft_spans), "craft-labels:", len(craft_labels))


label_list = ["O", "B-Chemical", "I-Chemical", "B-role", "I-role"]

id2label = {
        0: "O",
        1: "B-chemical",
        2: "I-chemical",
        3: "B-role",
        4: "I-role"
    }
    
label2id = {
        "O": 0,
        "B-chemical": 1,
        "I-chemical": 2,
        "B-role": 3,
        "I-role": 4
    }

from ModelTuner import ModelTuner
tuner = ModelTuner("google/electra-base-discriminator", label_list, id2label, label2id)
tuner.align_token_data(cdr_spans, cdr_labels)
tuner.align_token_data(nlm_spans, nlm_labels)
# tuner.align_token_data(craft_spans, craft_labels)

cdr-spans: 9873 cdr-labels: 9873
nlm-spans: 29782 nlm-labels: 29782
craft-spans: 21638 craft-labels: 21638
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
tuner.train(num_train_epochs=30)

num_train_steps: 74340


Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForTokenClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForTokenClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/30
2230/2230 [==============================] - 275s 119ms/step - loss: 0.0870 - precision: 0.8792 - recall: 0.9009 - f1: 0.8899 - accuracy: 0.9880
Epoch 2/30
2230/2230 [==============================] - 266s 119ms/step - loss: 0.0315 - precision: 0.9079 - recall: 0.9202 - f1: 0.9140 - accuracy: 0.9901
Epoch 3/30
2230/2230 [==============================] - 265s 119ms/step - loss: 0.0204 - precision: 0.9120 - recall: 0.9449 - f1: 0.9282 - accuracy: 0.9908
Epoch 4/30
2230/2230 [==============================] - 266s 119ms/step - loss: 0.0151 - precision: 0.9210 - recall: 0.9428 - f1: 0.9318 - accuracy: 0.9921
Epoch 5/30
2230/2230 [==============================] - 265s 119ms/step - loss: 0.0100 - precision: 0.9291 - recall: 0.9447 - f1: 0.9368 - accuracy: 0.9924
Epoch 6/30
2230/2230 [==============================] - 266s 119ms/step - loss: 0.0078 - precision: 0.9324 - recall: 0.9442 - f1: 0.9383 - accuracy: 0.9924
Epoch 7/30
2230/2230 [==============================] - 265s 119

In [7]:
from IPython.display import display, HTML
tuner.load_model("./model/chemical_extract_google-electra-base-discriminator")

html = tuner.infer_html("""To a suspension of bis(2-chloroethyl)amine hydrochloride (4.68 g, 26.2 mmol) in dry THF (50 mL) was added Et3N (18.3 mL, 131 mmol) at 0 °C and the mixture was stirred for 15 min. p-Toluenesul-fonyl chloride (5.00 g, 26.25 mmol) and DMAP (a spatula pinch) were added. The reqaction mixture was allowed to warm to room temperature and was stirred overnight. When TLC analysis showed complete conversion, the mixture was ﬁltered to remove the Et3N4HCl and extracted with EtOAc. The combined organic phases were washed with brine, dried (Na2SO4), ﬁltered, and concentrated. The residue was puriﬁed by ﬂash column chromatography to afford N,N-bis(2-chlo-roethyl)-4-methylbenzenesulfonamide (6.22 g,80% yield).""")
display(HTML(html))


Some layers from the model checkpoint at ./model/chemical_extract_google-electra-base-discriminator were not used when initializing TFElectraForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraForTokenClassification were initialized from the model checkpoint at ./model/chemical_extract_google-electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForTokenClassification for predictions without further tra

In [8]:
html = tuner.infer_html("""Deoxygenation of nitrous oxide (N2O) has significant environmental implications as it is not only a potent greenhouse gas but is also the main substance responsible for the depletion of ozone in the stratosphere.""")
display(HTML(html))

In [9]:
import spacy
nlp = spacy.load("en_core_web_lg")

text = craft.text[17069463]
doc = nlp(text)
for s in doc.sents:
    html = tuner.infer_html(s.text)
    display(HTML(html))

NameError: name 'craft' is not defined